<h1 align="center">Sentiment Analysis with LSTMs</h1>

First, we need to create word vectors. For simplicity, we will use a pretrained Word2Vec model
with Ukrainian words-vectors, each of which has a dimension of 300.

This sample of word vectors was created on the basis of fiction literature. We chose the lematized version of this model because we already have our sample, which we processed in the previous part, which would fit perfectly here.

The model can be found on [this website](https://lang.org.ua/uk/models/).

To get started, let's download the necessary libraries:

In [1]:
import numpy as np
import pandas as pd
import pickle
import gensim, logging
import gensim.models.keyedvectors as word2vec
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will load the sample data we processed in the previous part:

In [2]:
with open('documents.pql', 'rb') as f:
     docs = pickle.load(f)

In [3]:
print(docs[2])

(['працювати', 'автор', 'секція', 'наркоманія', 'чиказький', 'поліцейський', 'управління', 'бути', 'ласка', 'дозволяти', 'сприйняття', 'правда', 'автор', 'думка', 'чиказький', 'відділ', 'поліція', 'чоловік', 'жінка', 'гордо', 'служити', 'автор', 'стверджувати', 'секція', 'боротьба', 'наркотик', 'політик', 'сприяти', 'занепад', 'думати', 'процвітати', 'практик', 'використовувати', 'користь', 'бачити', 'егоїстичний', 'незрілий', 'юнак', 'думати', 'перший', 'якщо', 'автор', 'вважати', 'командир', 'наркологічний', 'секція', 'огидний', 'вживання', 'наркотик', 'невдалий', 'аналіз', 'сеча', 'помилятися', 'якщо', 'читати', 'книга', 'ласка', 'звинувачувати', 'неприємний', 'історія', 'людина'], 'neg')


In [4]:
print("Number of documents:", len(docs))

Number of documents: 2000


Now we will load our word2vec model. 

This may take some time, as the model contains approximately 58 500 words, so we will get a 58500 x 300 embedding matrix that contains all the values of the word vectors.

In [5]:
model = word2vec.KeyedVectors.load_word2vec_format('fiction.lowercased.lemmatized.word2vec.300d.txt', binary=False)

Now let's get a list of all the words from our dictionary:

In [6]:
words = list(model.vocab)

Just to make sure everything is loaded correctly, we can look at the dimensions of the dictionary list and the embedding matrix:

In [7]:
print(words[:100], "\n\nTotal words:", len(words), "\n\nWord-Vectors shape:", model.vectors.shape)

['і', 'не', 'на', 'в', 'що', 'з', 'а', 'я', 'й', 'він', 'до', 'бути', 'як', 'у', 'та', 'вона', 'його', 'за', 'ти', 'вони', 'це', 'свій', 'той', 'так', 'ж', 'ми', 'ви', 'то', 'але', 'ще', 'такий', 'по', 'її', 'себе', 'могти', 'вже', 'який', 'один', 'сам', 'все', 'про', 'сказати', 'мати', 'чи', 'від', 'знати', 'рука', 'око', 'мій', 'коли', 'те', 'тільки', 'щоб', 'ні', 'цей', 'всі', 'тут', 'бо', 'там', 'б', 'хто', 'де', 'для', 'голова', 'казати', 'тепер', 'люди', 'під', 'хотіти', 'i', 'и', 'із', 'бачити', 'стояти', 'о', 'піти', 'земля', 'щось', 'час', 'якийсь', 'наш', 'слово', 'тоді', 'треба', 'булий', 'великий', 'аж', 'раз', 'перед', 'хоч', 'над', 'би', 'навіть', 'хата', 'пан', 'уже', 'ну', 'дивитися', 'йти', 'думати'] 

Total words: 58492 

Word-Vectors shape: (58492, 300)


We can also find a word like "гарний" in our word list and then access the corresponding vector through the embedding matrix:

In [8]:
print(model['гарний'])

[-0.076125  0.553508 -0.843871  0.210367 -1.051    -0.456979 -0.071229
  0.295579 -0.239752 -1.436497  1.411778  1.076445 -1.785111 -0.305013
 -0.775204 -1.181594 -0.742579  0.024011 -1.269905 -0.540188  0.451396
  0.782721 -0.309379 -1.417217  0.702385 -1.744658  0.032138 -0.098305
 -0.99159  -0.109905  0.069233  0.37022  -0.247679  0.632742 -0.693778
  0.076653  0.531913 -1.158516  0.188859 -0.357537  0.296323  1.144795
  0.458457 -0.693163 -0.268378  0.105673  0.170314  1.165459  0.258425
  0.550712 -0.324439  0.132404 -0.201105 -0.525112  0.103556 -1.297686
 -0.550728 -0.309694 -0.371412  0.007037 -0.263882  0.096639 -0.861688
  0.504681 -0.304385  0.842935  0.014222  0.039818 -0.442853 -0.848775
 -0.329982  0.096731 -0.132125 -0.258058  0.65424  -0.669513  0.765716
 -0.695802 -0.299159 -0.648143  2.504469  0.17683  -0.698639  0.415211
 -0.021573  0.740519 -0.163555  0.39969  -0.410044  0.133693 -0.452481
 -1.250075  1.471488  0.030436  0.728958  1.194957  0.483038  0.620524
 -0.07